# aksty/promptgen: Prompt generation for Text-to-Image Models

Huggingface model : [aksty/promptgen](https://huggingface.co/aksty/promptgen)

This is a text generation model trained on data specifically designed to generate prompts for text-to-image models. It is based on the [EleutherAI/gpt-neo-125M](https://huggingface.co/EleutherAI/gpt-neo-125M) pre-trained model, which has been fine-tuned using the [Gustavosta/Stable-Diffusion-Prompts](https://huggingface.co/datasets/Gustavosta/Stable-Diffusion-Prompts) dataset.
![image](https://huggingface.co/aksty/promptgen/resolve/main/image.jpg)


## Usage

To use this model, you will need to have `PyTorch` and the `transformers` library installed. You can then use the following code to generate text using the model:

```python
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM
tokenizer = GPT2Tokenizer.from_pretrained("aksty/promptgen")
model = GPTNeoForCausalLM.from_pretrained("aksty/promptgen")
def generate_text(prompt):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  outputs = model.generate(input_ids, do_sample=True, max_length=100)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)
```

Output :
```python
generate_text("A painting of an ancient city ")
```
```
['A painting of an ancient city  on the top of a cliff, a small sign charging through the sky, cinematic view, epic sky, detailed, concept art, low angle, high detail, warm lighting, volumetric, godrays, vivid, beautiful, trending on artstation, by jordan grimmer, huge scene, grass, art greg rutkowski']
```

## Disclaimer
It is important to note that the results generated by promptgen are not guaranteed to be accurate, complete, or suitable for any particular purpose. The model is intended for research and educational purposes only and should not be relied upon for any other purposes. The generated text may contain errors, omissions, or inappropriate language. The user of the model is solely responsible for any actions or decisions made based on the generated text.




In [ ]:
#@title Install Dependencies
!pip install diffusers
!pip install transformers
!pip install accelerate
!pip install scipy
!pip install gradio

In [12]:
#@title Import Dependencies
# Import dependencies
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM
import gradio as gr
import numpy as np
from PIL import Image
import uuid

# Create folder for saving images
!mkdir saved_images

grTitle = """
# promptgen -> stable diffusion v2
Huggingface model : [aksty/promptgen](https://huggingface.co/aksty/promptgen)

This is a text generation model trained on data specifically designed to generate prompts for text-to-image models. It is based on the [EleutherAI/gpt-neo-125M](https://huggingface.co/EleutherAI/gpt-neo-125M) pre-trained model, which has been fine-tuned using the [Gustavosta/Stable-Diffusion-Prompts](https://huggingface.co/datasets/Gustavosta/Stable-Diffusion-Prompts) dataset.

## Disclaimer
It is important to note that the results generated by promptgen are not guaranteed to be accurate, complete, or suitable for any particular purpose. The model is intended for research and educational purposes only and should not be relied upon for any other purposes. The generated text may contain errors, omissions, or inappropriate language. The user of the model is solely responsible for any actions or decisions made based on the generated text.

"""

mkdir: cannot create directory ‘saved_images’: File exists


In [ ]:
#@title Initialize models
# Initialize stable diffusion v2
model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Initialize aksty/promptgen
tokenizer = GPT2Tokenizer.from_pretrained("aksty/promptgen")
model = GPTNeoForCausalLM.from_pretrained("aksty/promptgen")


In [ ]:
#@title Run gradio
import gradio as gr
import uuid

steps = "25" #@param [25,50,75,100]
number_of_images = "5" #@param [1,5,10,15,20]
width_and_height = "512" #@param [512,768]

width = width_and_height
height = width_and_height

grtable = f"""
| Parameter | Value |
|--|--|
|  Steps|  {steps} |
|  Number of images| {number_of_images} |
|  Width x Height| {width} x {height} |
"""



# Function to generate prompt
def process_text(prompt):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  outputs = model.generate(input_ids, do_sample=True, max_length=100)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Function to generate image using stable diffusion v2
def generate_image(prompt):
  images = pipe(
      prompt,
      height = int(height),
      width = int(width),
      num_images_per_prompt = int(number_of_images),
      num_inference_steps = int(steps),
     ).images
  prompt = prompt.replace(",","_").replace(" ","_").strip()
  prompt = prompt[0:10] 
  for img in images:
    img.save(f'saved_images/{prompt}_{str(uuid.uuid1())}.jpg')
  return images


# ui
with gr.Blocks() as interface:
    gr.Markdown(grTitle)
    gr.Markdown(grtable)
    with gr.Row():
        inp = gr.Textbox(label="Prompt",placeholder="Enter the Prompt",lines=10)
        img = gallery = gr.Gallery(label="Generated images").style(grid=[2], height="auto")
    
    with gr.Row():
      generatePromptbtn = gr.Button("Generate Prompt")
      submitWithPromptbtn = gr.Button("Generate Image")

    generatePromptbtn.click(fn=process_text, inputs=inp, outputs=inp)
    submitWithPromptbtn.click(fn=generate_image,inputs=inp,outputs=img)

# Launch UI in debug mode
interface.launch(debug=True)